# Using InlineAgent Runnable with Return of Control (RoC)

The following examples will show how you can begin integrating with the InlineAgent Runnable.

### Prerequisites:
1. Set your [aws credentials](https://docs.aws.amazon.com/cli/v1/userguide/cli-configure-envvars.html#envvars-set) for your environment, example.
2. Ensure that langchain, langgraph are installed in the environment and that the local langchain-aws is accessible from the path or installed into the environment. 

## Example 1: Create a mortgage agent that determines the interest rate
In this example, we create a mortgage agent with two tools. The first tool will return the asset values of a given asset holder. The second tool will return the interest rate for a given asset holder with a given asset value.

### Step 1: Define the tools for the agent

In [1]:
from langchain_core.tools import tool
from langchain_aws.agents import BedrockInlineAgentsRunnable
from langchain_core.messages import HumanMessage


@tool("AssetDetail::getAssetValue")
def get_asset_value(asset_holder_id: str) -> str:
    """
    Get the asset value for an owner id

    Args:
        asset_holder_id: The asset holder id

    Returns:
        The asset value for the given asset holder
    """
    return f"The total asset value for {asset_holder_id} is 100K"


@tool("AssetDetail::getMortgageRate")
def get_mortgage_rate(asset_holder_id: str, asset_value: str) -> str:
    """
    Get the mortgage rate based on asset value

    Args:
        asset_holder_id: The asset holder id
        asset_value: The value of the asset

    Returns:
        The interest rate for the asset holder and asset value
    """
    return f"The mortgage rate for {asset_holder_id} with asset value of {asset_value} is 8.87%"


tools = [get_asset_value, get_mortgage_rate]
print(tools)

[StructuredTool(name='AssetDetail::getAssetValue', description='Get the asset value for an owner id\n\nArgs:\n    asset_holder_id: The asset holder id\n\nReturns:\n    The asset value for the given asset holder', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getAssetValue'>, func=<function get_asset_value at 0x106f3f880>), StructuredTool(name='AssetDetail::getMortgageRate', description='Get the mortgage rate based on asset value\n\nArgs:\n    asset_holder_id: The asset holder id\n    asset_value: The value of the asset\n\nReturns:\n    The interest rate for the asset holder and asset value', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getMortgageRate'>, func=<function get_mortgage_rate at 0x106f3d4e0>)]


### Step 2: Define the agent configuration and create the runnable

In [2]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
instructions = (
    "You are a friendly and cheerful assistant that answers questions valiantly."
)
inline_agent_config = {
    "foundation_model": foundation_model,
    "instruction": instructions,
    "tools": tools,
    "enable_trace": False,
}

runnable = BedrockInlineAgentsRunnable.create(
    region_name="us-west-2", inline_agent_config=inline_agent_config
)

print("BedrockInlineAgentsRunnable created successfully.")
messages = [HumanMessage(content="what is my mortgage rate for id AVC-1234")]


### Step 3: Invoke the agent

In [3]:
# Invoke the inlineAgent
output = runnable.invoke(messages)
print(output)

## Example 2: Updating the configs on the fly
* You can modify any agent configuration for each request by providing the updates in the `inline_agent_configuration`. 
* You can also start a new conversation by providing a `session_id`

**Note:** Updates are additive and will build on previous configurations, if you want to remove a config, pass in a `NoneType`

In [4]:
inline_agent_config = {
    "instruction": "You are now a pirate that answers questions in Haiku! You MUST end all sentences in 'Yarr!'",
    "tools": None,
    "enable_trace": True,
}
output = runnable.invoke(
    [HumanMessage(content="What are cars?")],
    inline_agent_config=inline_agent_config,
    session_id="test-sesh-1",
)
print(output)

## Example 3: Using CodeInterpretter functionality

The code interpretation (CI) enables your agent to generate, run, and troubleshoot your application code in a secure test environment. You can enable CI on an Inline Agent by passing in `enable_code_interpreter = True` while creating the runnable or making the invoke request.

In [5]:
from langchain_core.tools import tool
from langchain_aws.agents import BedrockInlineAgentsRunnable
from langchain_core.messages import HumanMessage

foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
instructions = "You are an agent who helps with getting the mortgage rate based on the current asset valuation"
inline_agent_config = {
    "foundation_model": foundation_model,
    "instruction": instructions,
    "enable_code_interpreter": True,
    "enable_trace": False,
}

runnable = BedrockInlineAgentsRunnable.create(
    region_name="us-west-2", inline_agent_config=inline_agent_config
)

print("BedrockInlineAgentsRunnable and AgentExecutor created successfully.")

# Invoke the agent.
output = runnable.invoke(
    [
        HumanMessage(
            content="Can you please help plot a line graph for the following values - [1,199,1999, 199999, 122] against an increasing x value [1,2,3,4,5]?"
        )
    ]
)

print("Agent Output:")
print(output)